# 수소(H) + 헬륨-4(He-4) 빅뱅 핵합성(BBN) 시뮬레이션 구현해보기

이 과제는 **중학교 수학/과학 지식**으로 빅뱅 핵합성의 기본 구조를 이해하고, 이를 **파이썬 코드로 구현**해 보는 것을 목표로 한다.

---

## 오늘의 질문(3개)

1. 왜 고온에서는 $n/p$가 "평형"을 따라가나?  
2. 왜 어떤 시점부터 평형이 깨지고 "동결(freeze-out)"되나?  
3. 왜 마지막에 $Y_p \approx 2X_n$이 되는가? (보존/개수 세기)

---

```otter
# ASSIGNMENT CONFIG
name: bbn_toy_assignment
runs_on: colab
tests:
  files: true
```

In [ ]:
# (Colab 권장) Otter 설치 + tests 확인 + 채점기 준비
# - tests 폴더가 없으면 즉시채점이 작동하지 않는다.
# - 학교/수업 안내에 따라, 먼저 저장소를 '클론'한 뒤 실행하자.

import os, sys

# Colab에서만 설치 (로컬에 이미 있으면 건너뜀)
try:
    import otter
except Exception:
    !pip -q install otter-grader
    import otter

assert os.path.isdir("tests"), "tests 폴더가 없습니다. 저장소를 제대로 클론했는지 확인하세요."
grader = otter.Notebook()

print("✅ 준비 완료: grader.check('qA1') 처럼 즉시채점을 실행할 수 있어요.")

In [ ]:
# Helper: ensure answer variables exist before grading
import re, os

def _test_vars_from_file(test_name):
    p = os.path.join('tests', test_name + '.py')
    if not os.path.exists(p):
        return set()
    with open(p, 'r', encoding='utf-8') as f:
        txt = f.read()
    return set(re.findall(r"(ans_[A-Za-z0-9_]+)", txt))

# Safely wrap the original grader.check (avoid recursion)
orig_check = getattr(grader, '_orig_check', None) or getattr(grader, 'check')
def safe_check(q, *a, **k):
    missing = [n for n in _test_vars_from_file(q) if n not in globals()]
    if missing:
        print('채점 전 정의되지 않은 변수:', missing)
        for n in missing:
            globals()[n] = None
    # ensure we pass notebook globals as the global_env without duplicating the kwarg
    k.pop('global_env', None)
    k['global_env'] = globals()
    return orig_check(q, *a, **k)

# Replace grader.check with safe_check once
if not hasattr(grader, '_safe_check_installed'):
    grader._orig_check = orig_check
    grader.check = safe_check
    grader._safe_check_installed = True

# Part A. 개념 이해(자동채점) 

- 답은 가능한 한 **O/X**, **A/B/C**, 또는 **짧은 숫자**로 끝난다.
- 로그축은 "로그를 계산"하지 않는다. 눈금이 "10배씩"이라는 규칙으로 읽기만 한다.

## A0. 플라즈마와 초기 우주의 온도

빅뱅 직후 우주는 극도로 뜨거워서 원자가 존재할 수 없었다. 온도가 충분히 높으면 전자가 원자핵에 붙어 있지 못하고 분리된 상태(**플라즈마**)가 된다.

**핵심 개념:**
- 열에너지 $kT$와 결합에너지의 비교로 상태를 판단
- 우주 팽창 → 온도 하강 (반비례 관계: $T \propto 1/a$, $a$는 우주 크기)

In [ ]:
# 볼츠만 상수 (eV/K 단위)
k_B_eV = 8.617e-5  # eV/K

# 수소 원자의 이온화 에너지 (전자를 떼어내는 데 필요한 에너지)
E_ionization_H = 13.6  # eV

# --- A0_1: 온도 T = 10억 K (1e9 K)에서 열에너지 kT를 eV 단위로 계산하라 ---
T_hot = 1e9  # K
kT_hot = ...  # 빈칸


ans_A0_3 = ...  #@param ["O","X"]  # T ∝ 1/a 이므로 팽창하면 온도는 낮아진다

# --- A0_4: 우주가 100배 팽창하면 온도는 처음의 몇 배가 되는가? ---
expansion_factor = 100
ans_A0_4 = ...  # 빈칸

In [ ]:
grader.check('qA0', global_env=globals())

## A1. 비와 비율 — $X_n$과 $n/p$ 변환

중성자 분율 $X_n$과 중성자-양성자 비 $n/p$는 서로 변환 가능하다:

$$
X_n = \frac{n}{n+p}, \quad \frac{n}{p} = \frac{X_n}{1-X_n}
$$

**주의:** 이 관계는 **비선형**이다. $X_n$이 2배가 된다고 $n/p$도 2배가 되지 않는다!

In [ ]:
# --- A1_1: Xn에서 n/p로 변환하는 함수를 완성하라 ---
def Xn_to_np_ratio(Xn):
    """Xn = n/(n+p) 일 때 n/p를 반환"""
    return ...  # 빈칸

# --- A1_2: Xn이 0.10에서 0.125로 증가하면 n/p는 몇 % 증가하는가? ---
np_before = Xn_to_np_ratio(0.10)
np_after = Xn_to_np_ratio(0.125)
ans_A1_2 = ...  # 빈칸

# --- A1_4 (역산): n/p = 1/7 일 때 Xn은 얼마인가? ---
# 힌트: n/p = Xn/(1-Xn) = 1/7 → Xn = ?
ans_A1_4 = ...  # 빈칸

In [ ]:
grader.check('qA1', global_env=globals())

## A2. 핵반응의 보존법칙

핵반응에서는 **질량수(A)**와 **전하(Z)**가 보존된다.

예: $D + D \to {}^3\text{He} + n$ 반응에서
- 반응 전 질량수: $2 + 2 = 4$
- 반응 후 질량수: $3 + 1 = 4$ ✓

**주의:** 질량수는 보존되지만, 실제 **질량**은 결합에너지 차이로 약간 달라진다 (E=mc²).

In [ ]:
# 핵종 정보: (양성자 수 Z, 중성자 수 N) → 질량수 A = Z + N
nuclei = {
    'p': (1, 0),    # 양성자
    'n': (0, 1),    # 중성자
    'D': (1, 1),    # 중수소
    'He3': (2, 1),  # 헬륨-3
    'He4': (2, 2),  # 헬륨-4
}

def mass_number(name):
    """핵종의 질량수 A = Z + N"""
    Z, N = nuclei[name]
    return Z + N

def charge(name):
    """핵종의 전하 (= 양성자 수 Z)"""
    Z, N = nuclei[name]
    return Z

# --- A2_1: p + n → D 반응에서 질량수 보존을 확인하라 ---
reactants_A = mass_number('p') + mass_number('n')
products_A = mass_number('D')
ans_A2_1 = ...  # 빈칸: reactants_A == products_A (True)

# --- A2_2: D + D → He3 + n 반응에서 전하 보존을 확인하라 ---
reactants_Z = charge('D') + charge('D')
products_Z = ...  # 빈칸: charge('He3') + charge('n')
ans_A2_2 = (reactants_Z == products_Z)


ans_A2_3 = ...  #@param ["O","X"]  # 둘 다 양성자 2개 (같은 원소, 다른 동위원소)


ans_A2_4 = ...  #@param ["O","X"]  # 질량수는 보존되지만 실제 질량은 결합에너지 때문에 다름

In [ ]:
grader.check('qA2', global_env=globals())

## A3. 성분비와 제한 반응물 — $Y_p = 2X_n$

He-4 하나를 만들려면 **양성자 2개 + 중성자 2개**가 필요하다.
초기 우주에서 중성자가 양성자보다 훨씬 적으므로, **중성자가 제한 반응물**이 된다.

$$
Y_p \approx 2X_n(T_{\mathrm{nuc}})
$$

**시나리오 문제:** 만약 일부 중성자가 He-4 대신 중수소(D)로 갔다면?

In [ ]:
# --- A3_1: Xn = 0.12일 때 Yp = 2*Xn을 계산하라 ---
Xn_nuc = 0.12
ans_A3_1 = ...  # 빈칸

# --- A3_2 (시나리오): 중성자의 10%가 D로, 90%가 He-4로 갔다면 Yp는? ---
# He-4의 질량 기여: 4 * (중성자 90% / 2) = 2 * 0.9 * Xn
# D의 질량 기여: 2 * (중성자 10% / 1) = 2 * 0.1 * Xn
# Yp(He만) = 2 * 0.9 * Xn = ?
Xn_scenario = 0.12
fraction_to_He4 = 0.9
ans_A3_2 = ...  # 빈칸


ans_A3_3 = ...  #@param ["O","X"]

# --- A3_4: 관측된 우주의 He 질량비는 약 0.24이다. Xn은 약 얼마였겠는가? ---
Yp_observed = 0.24
ans_A3_4 = ...  # 빈칸


ans_A3_5 = ...  #@param ["O","X"]

In [ ]:
grader.check('qA3', global_env=globals())

## A4. 연립방정식으로 수소/헬륨 비율 구하기

전체 핵자수를 1로 정규화하면:
- 핵자수 보존: $1 = N_H + 4 \cdot N_{He}$
- 중성자 보존: $X_n = 2 \cdot N_{He}$

이 연립방정식을 풀어 $N_H$와 $N_{He}$를 구할 수 있다.

**역산 문제:** $N_{He}$가 주어지면 $X_n$을 역으로 구할 수 있는가?

In [ ]:
# --- A4_1: 연립방정식을 푸는 함수를 완성하라 ---
def solve_NH_NHe(Xn):
    """Xn이 주어지면 N_H, N_He를 반환"""
    N_He = ...  # 빈칸
    return N_H, N_He

# 테스트: Xn = 0.10
N_H_test, N_He_test = solve_NH_NHe(0.10)
# 결과를 확인해보세요

# --- A4_2 (역산): N_He = 0.06이면 Xn은? ---
N_He_given = 0.06
ans_A4_2 = ...  # 빈칸

# --- A4_3 (역산): N_H = 0.76이면 N_He와 Xn은? ---
N_H_given = 0.76
N_He_from_NH = ...  # 빈칸
ans_A4_3 = ...  # 빈칸


ans_A4_4 = ...  #@param ["O","X"]  # 과결정(overdetermined) - 모순이 생길 수 있음

In [ ]:
grader.check('qA4', global_env=globals())

## A5. 반감기와 지수감쇠

자유 중성자는 불안정하여 붕괴한다. 반감기 $t_{1/2} \approx 10$분.

$$
N(t) = N_0 \cdot \left(\frac{1}{2}\right)^{t/t_{1/2}} = N_0 \cdot e^{-t/\tau}
$$

여기서 $\tau = t_{1/2} / \ln 2 \approx 14.4$분 (평균 수명).

**핵심:** 지수감쇠는 "같은 시간마다 같은 **비율**로 감소"한다 (같은 **양**이 아님!).

In [ ]:
import math

# --- A5_1: 반감기 후 남은 비율을 계산하는 함수 ---
def remaining_fraction(n_half_lives):
    """n번의 반감기 후 남은 비율"""
    return ...  # 빈칸

# --- A5_3 (역산): 40분 후 1/16이 남았다면 반감기는 몇 분? ---
# (1/2)^(40/t_half) = 1/16 = (1/2)^4 → 40/t_half = 4
ans_A5_3 = ...  # 빈칸


ans_A5_4 = ...  #@param ["O","X"]


ans_A5_5 = ...  #@param ["O","X"]

In [ ]:
grader.check('qA5', global_env=globals())

## A6. 거듭제곱과 스케일링

물리량들이 온도 $T$의 거듭제곱으로 변한다:
- 시간: $t \propto T^{-2}$ (온도가 내려가면 시간이 흐른다)
- 반응률: $\Gamma \propto T^5$ (온도가 내려가면 급격히 감소)

**역산 문제:** $\Gamma$가 100배 감소하려면 $T$는 몇 배 변해야 하는가?

In [ ]:
# --- A6_1: T가 1/10배가 되면 t = A/T² 는 몇 배가 되는가? ---
T_ratio = 1/10  # T가 1/10배
t_ratio = ...   # 빈칸

# --- A6_2: T가 1/10배가 되면 Γ = γ₀T⁵ 는 몇 배가 되는가? ---
Gamma_ratio = ...  # 빈칸

# --- A6_3 (역산): Γ가 32배 감소하려면 T는 몇 배가 되어야 하는가? ---
# Γ_new / Γ_old = (T_new/T_old)^5 = 1/32 = (1/2)^5
ans_A6_3 = ...  # 빈칸

# --- A6_4 (역산): t가 100배 증가했다면 T는 몇 배가 되었는가? ---
# t ∝ 1/T² → t_new/t_old = (T_old/T_new)² = 100 → T_new/T_old = 1/10
ans_A6_4 = ...  # 빈칸

# --- A6_5: 10⁹ K는 10⁸ K의 몇 배인가? ---
ans_A6_5 = ...  # 빈칸

In [ ]:
grader.check('qA6', global_env=globals())

## A7. 무차원비와 기준선

$\Gamma/H$는 **무차원비**(단위 없는 비)이다:
- $\Gamma$: 반응률 [1/초]
- $H$: 허블 팽창률 [1/초]
- $\Gamma/H$: 단위 없음 → 1과 직접 비교 가능

**기준선:** $\Gamma/H = 1$은 "반응과 팽창이 같은 속도"인 임계점이다.

In [ ]:
# --- A7_1: Γ = 100 [1/s], H = 10 [1/s] 일 때 Γ/H는? ---
Gamma = 100  # 1/s
H = 10       # 1/s
ans_A7_1 = ...  # 빈칸

# --- A7_2: Γ/H = 10이면 반응과 팽창 중 어느 쪽이 우세한가? ---
# A: 반응이 우세 (Γ > H)
# B: 팽창이 우세 (Γ < H)
ans_A7_2 = ...  #@param ["A","B"]

# --- A7_3: Γ/H가 10에서 0.1로 변하는 동안 H가 2배가 되었다면, Γ는 몇 배가 되었는가? ---
# Γ/H: 10 → 0.1 (100배 감소)
# H: 1 → 2 (2배 증가)
# Γ를 계산해보세요
ratio_initial = 10
ratio_final = 0.1
H_ratio = 2  # H가 2배
ans_A7_3 = ...  # 빈칸


ans_A7_4 = ...  #@param ["O","X"]

In [ ]:
grader.check('qA7', global_env=globals())

## A8. 그래프 읽기와 오일러 방법

시뮬레이션에서 다음 값을 구하는 기본 공식:
$$
x_{\text{next}} = x + (\text{변화율}) \times dt
$$

**그래프 해석:**
- 실선이 점선(평형)을 따라가면: 반응이 빠름 ($\Gamma/H > 1$)
- 실선이 점선에서 벗어나면: 동결 시작 ($\Gamma/H < 1$)

In [ ]:
# --- A8_1: 오일러 방법으로 다음 값 계산 ---
x_current = 0.15
rate = -0.01  # 감소율 (음수)
dt = 10       # 시간 간격
x_next = ...  # 빈칸

# --- A8_2: 변화량 Δx를 계산하라 ---
ans_A8_2 = ...  # 빈칸

# --- A8_3: dt가 2배가 되면 변화량 Δx는 몇 배? ---
ans_A8_3 = ...  # 빈칸


ans_A8_5 = ...  #@param ["O","X"]

In [ ]:
grader.check('qA8', global_env=globals())

## A9. 모형과 근사

이 과제의 시뮬레이션은 **토이 모델**이다:
- 실제 우주를 단순화한 근사
- 정밀한 예측이 아닌 **구조 이해**가 목적
- 결과는 "대략적인" 값으로 해석

**보간(interpolation)**: 표에서 중간값을 추정하는 방법

In [ ]:
# --- A9_1: 선형 보간 함수 완성 ---
def linear_interpolate(x, x0, x1, y0, y1):
    """x0에서 y0, x1에서 y1일 때, x에서의 y값을 선형 보간"""
    # y = y0 + (y1 - y0) * (x - x0) / (x1 - x0)
    return ...  # 빈칸 완성

# 테스트: T=5에서 T=3 사이, T=4에서의 Xn은?
# T=5일 때 Xn=0.20, T=3일 때 Xn=0.10
ans_A9_1 = linear_interpolate(4, 5, 3, 0.20, 0.10)

# --- A9_2: 시뮬레이션 결과 Yp = 0.2468을 유효숫자 2자리로 반올림하면? ---
Yp_sim = 0.2468
ans_A9_2 = ...  # 빈칸


ans_A9_3 = ...  #@param ["O","X"]



ans_A9_4 = ...  #@param ["O","X"]

In [ ]:
grader.check('qA9', global_env=globals())

## A10. 약력, 핵력, 그리고 핵반응

BBN을 이해하려면 **자연의 기본 힘**과 **핵반응**에 대해 알아야 한다.

### 자연의 네 가지 기본 힘

| 힘 | 작용 대상 | 상대적 세기 | 도달 거리 |
|---|----------|-----------|----------|
| 강력(핵력) | 쿼크, 핵자 | 1 | 10⁻¹⁵ m (원자핵 크기) |
| 전자기력 | 전하를 가진 입자 | 10⁻² | 무한대 |
| **약력** | 모든 페르미온 | 10⁻⁵ | 10⁻¹⁸ m |
| 중력 | 모든 질량 | 10⁻⁴⁰ | 무한대 |

### 핵력 (강력)
- 양성자와 중성자를 원자핵 안에 **묶어두는** 힘
- 양성자끼리는 전기적으로 밀어내지만, 핵력이 더 강해서 핵이 안정
- **도달 거리가 매우 짧음** → 원자핵 크기 내에서만 작용

### 약력 (약한 상호작용)
- 입자의 **종류(flavor)**를 바꿀 수 있는 유일한 힘
- **중성자 ↔ 양성자 전환**을 담당!
- 중성자 붕괴: $n \to p + e^- + \bar{\nu}$ (베타 붕괴)
- 초기 우주에서 n ↔ p 평형을 유지시킴

### 중성자-양성자 전환 반응

**n → p 반응 (중성자 → 양성자):**
- $n + \nu \to p + e^-$ (중성미자 포획)
- $n + e^+ \to p + \bar{\nu}$ (양전자 포획)
- $n \to p + e^- + \bar{\nu}$ (자유 붕괴, 반감기 ~10분)

**p → n 반응 (양성자 → 중성자):**
- $p + e^- \to n + \nu$
- $p + \bar{\nu} \to n + e^+$

**핵심:** 고온에서는 이 반응들이 빨라서 평형 유지, 저온에서는 느려져서 "동결"

### 이 코드에서 다루는 핵반응

| 반응 | 설명 | 역할 |
|-----|------|-----|
| n ↔ p | 약력에 의한 전환 | n/p 비율 결정 |
| p + n → D | 중수소 생성 | 핵합성 시작 |
| D + D → He-3 + n | 헬륨-3 생성 | 중간 단계 |
| D + D → H-3 + p | 삼중수소 생성 | 중간 단계 |
| He-3 + n → He-4 + ... | 헬륨-4 생성 | 최종 생성물 |

In [ ]:
# --- A10_1: 약력의 특징 ---
# 약력은 입자의 종류(flavor)를 바꿀 수 있는 유일한 힘이다.
ans_A10_1 = ...  #@param ["O","X"]


ans_A10_2 = ...  #@param ["O","X"]

# --- A10_3: n → p 전환에 관여하는 힘은? ---
# A: 중력
# B: 전자기력
# C: 약력
ans_A10_3 = ...  #@param ["A","B","C"]


ans_A10_4 = ...  #@param ["O","X"]

# --- A10_5: 중성자가 양성자로 붕괴할 때 방출되는 입자는? ---
# A: 전자와 반중성미자 (e⁻ + ν̄)
# B: 양전자와 중성미자 (e⁺ + ν)
# C: 광자 (γ)
ans_A10_5 = ...  #@param ["A","B","C"]

# --- A10_6: BBN에서 대부분의 중성자가 최종적으로 들어가는 핵종은? ---
# A: 중수소 (D)
# B: 헬륨-4 (He-4)
# C: 리튬 (Li)
ans_A10_6 = ...  #@param ["A","B","C"]

In [ ]:
grader.check('qA10', global_env=globals())

# Part B. 미니 코딩 실습 — Part A 개념을 코드로 확인하기

Part A에서 배운 개념들을 **1~5줄의 짧은 코드**로 직접 구현해본다.
- Part A의 개념 이해 → Part B의 코드 구현 → Part C의 전체 시뮬레이션
- 여기서 만든 함수들은 Part C에서 재사용된다.

## B0. 온도와 에너지 (연결: A0) ★

플라즈마 상태인지 판단하려면 열에너지 $kT$를 계산해야 한다.

**공식:**
$$
kT = k_B \times T \quad \text{[eV]}
$$

여기서 $k_B = 8.617 \times 10^{-5}$ eV/K (볼츠만 상수)

In [ ]:
# --- B0_1: kT를 eV 단위로 계산하는 함수 ---
def kT_eV(T_kelvin):
    """온도 T(K)에서 열에너지 kT(eV)를 계산"""
    k_B = 8.617e-5  # eV/K
    return ...  # 빈칸
ans_B0_2 = kT_eV(1e8)  # 약 8617 eV

# --- B0_3: 온도비 계산 (팽창 전후) ---
T_before = 1e10  # K (팽창 전)
T_after = 1e9    # K (팽창 후)
ans_B0_3 = T_before / T_after  # 온도가 몇 배 감소했는가?

# --- B0_4: 우주가 100배 팽창했을 때 온도 변화 ---
# T ∝ 1/a 이므로 a가 100배 → T는 1/100배
expansion = 100
T_initial = 1e10  # K
ans_B0_4 = T_initial / expansion  # 최종 온도

In [ ]:
grader.check('qB0', global_env=globals())

## B1. 비율 변환 함수 (연결: A1) ★

$X_n$과 $n/p$ 사이의 **양방향 변환**을 함수로 구현한다.

**공식:**
$$
X_n \to \frac{n}{p}: \quad \frac{n}{p} = \frac{X_n}{1-X_n}
$$
$$
\frac{n}{p} \to X_n: \quad X_n = \frac{n/p}{1+n/p}
$$

In [ ]:
# --- B1_1: Xn → n/p 변환 (A1에서 이미 정의했으므로 복사) ---
def Xn_to_np(Xn):
    """Xn에서 n/p로 변환"""
    return Xn / (1 - Xn)

# --- B1_2: n/p → Xn 역변환 ---
def np_to_Xn(np_ratio):
    """n/p에서 Xn으로 역변환"""
    return ...  # 빈칸
np_val = Xn_to_np(Xn_orig)
Xn_back = np_to_Xn(np_val)
ans_B1_3 = abs(Xn_orig - Xn_back) < 1e-10

# --- B1_4: 테스트 - n/p = 0.2일 때 Xn은? ---
ans_B1_4 = np_to_Xn(0.2)  # 약 0.1667

In [ ]:
grader.check('qB1', global_env=globals())

## B2. 핵반응 보존 검증 (연결: A2) ★★

핵반응 전후에 **질량수(A)**와 **전하(Z)**가 보존되는지 코드로 검증한다.

**예시 반응:** $D + D \to {}^3\text{He} + n$

In [ ]:
# 핵종 데이터: (양성자 수 Z, 중성자 수 N)
nuclei_B2 = {
    'p': (1, 0),    # 양성자
    'n': (0, 1),    # 중성자
    'D': (1, 1),    # 중수소
    'He3': (2, 1),  # 헬륨-3
    'He4': (2, 2),  # 헬륨-4
}

# --- B2_1: 반응 보존 검증 함수 ---
def check_reaction(reactants, products):
    """반응 전후 질량수와 전하가 보존되는지 확인"""
    # 반응 전 질량수와 전하
    A_in = sum(nuclei_B2[r][0] + nuclei_B2[r][1] for r in reactants)
    Z_in = sum(nuclei_B2[r][0] for r in reactants)
    # 반응 후 질량수와 전하
    A_out = ...  # 빈칸
    return (A_in == A_out) and (Z_in == Z_out)

# --- B2_2: D + D → He3 + n 검증 ---
ans_B2_2 = check_reaction(['D', 'D'], ['He3', 'n'])

# --- B2_3: p + n → D 검증 ---
ans_B2_3 = check_reaction(['p', 'n'], ['D'])

# --- B2_4: 잘못된 반응 검증 (D → He4는 불가능) ---
ans_B2_4 = check_reaction(['D', 'D'], ['He4'])

In [ ]:
grader.check('qB2', global_env=globals())

## B3. 헬륨 질량비 계산 (연결: A3, A4) ★★

$Y_p = 2X_n$ 공식과 연립방정식을 코드로 구현한다.

**연립방정식:**
- 핵자수 보존: $1 = N_H + 4 \cdot N_{He}$
- 중성자 보존: $X_n = 2 \cdot N_{He}$

In [ ]:
# --- B3_1: Yp 계산 함수 ---
def Yp_from_Xn_B3(Xn):
    """Xn에서 헬륨 질량비 Yp 계산"""
    return 2 * Xn

# --- B3_2: 연립방정식 풀기 ---
def solve_abundances(Xn):
    """핵자수 1로 정규화시 N_H, N_He 반환"""
    N_He = Xn / 2
    N_H = ...  # 빈칸
N_H, N_He = solve_abundances(Xn_test)
ans_B3_3 = abs(N_H + 4*N_He - 1.0) < 1e-10

# --- B3_4: Yp와 N_He의 관계 확인 ---
# Yp = 4 * N_He (헬륨의 질량비 = 4 × 헬륨 원자핵 수)
Yp_calc = Yp_from_Xn_B3(Xn_test)
Yp_from_N_He = 4 * N_He
ans_B3_4 = abs(Yp_calc - Yp_from_N_He) < 1e-10

In [ ]:
grader.check('qB3', global_env=globals())

## B4. 지수감쇠 함수 (연결: A5) ★★

반감기와 평균수명 변환, 지수감쇠 계산을 함수로 구현한다.

**공식:**
$$
\tau = \frac{t_{1/2}}{\ln 2}, \quad N(t) = N_0 \cdot e^{-t/\tau}
$$

In [ ]:
import math

# --- B4_1: 반감기 → 평균수명 변환 ---
def halflife_to_tau(t_half):
    """반감기를 평균수명으로 변환"""
    return t_half / math.log(2)

# --- B4_2: 지수감쇠 후 남은 비율 ---
def decay_fraction(t, tau):
    """시간 t 후 남은 비율 (평균수명 tau 사용)"""
    return ...  # 빈칸
tau = halflife_to_tau(t_half)
fraction_at_halflife = decay_fraction(t_half, tau)
ans_B4_3 = abs(fraction_at_halflife - 0.5) < 0.01

# --- B4_4: 중성자 반감기 적용 (τ_n ≈ 879.4초) ---
tau_neutron = 879.4  # 초
t_elapsed = 300  # 초 (5분)
ans_B4_4 = decay_fraction(t_elapsed, tau_neutron)  # 약 0.71

In [ ]:
grader.check('qB4', global_env=globals())

## B5. 거듭제곱 스케일링 (연결: A6) ★★

시간-온도 관계와 반응률-온도 관계를 함수로 구현한다.
이 함수들은 **Part C에서 그대로 사용**된다!

**공식:**
$$
t(T) = \frac{A}{T^2}, \quad \Gamma(T) = \gamma_0 T^5
$$

In [ ]:
# --- B5_1: 시간-온도 관계 ---
def t_of_T_B5(T, A=0.738):
    """t = A / T² [초]"""
    return ...  # 빈칸
    """Γ = γ₀ × T⁵ [1/초]"""
    return ...  # 빈칸
t_ratio_B5 = t_of_T_B5(T2) / t_of_T_B5(T1)  # t는 4배 증가해야 함
Gamma_ratio_B5 = Gamma_of_T_B5(T2) / Gamma_of_T_B5(T1)  # Γ는 1/32배

ans_B5_3 = abs(t_ratio_B5 - 4) < 0.1
ans_B5_4 = abs(Gamma_ratio_B5 - 1/32) < 0.01

In [ ]:
grader.check('qB5', global_env=globals())

## B6. 무차원비와 오일러 스텝 (연결: A7, A8) ★★★

$\Gamma/H$ 무차원비 계산과 오일러 방법을 구현한다.
평형으로 완화하는 핵심 공식도 여기서 연습한다.

**핵심 공식:**
$$
X_{\mathrm{temp}} = X_{\mathrm{eq}} + (X - X_{\mathrm{eq}}) \cdot e^{-\Gamma \cdot dt}
$$

In [ ]:
# --- B6_1: Γ/H 무차원비 계산 ---
def Gamma_over_H(Gamma, H):
    """무차원비 = 반응률 / 팽창률"""
    return ...  # 빈칸: Gamma / H

# --- B6_2: 오일러 스텝 ---
def euler_step(x, rate, dt):
    """x_next = x + rate * dt"""
    return ...  # 빈칸: x + rate * dt

# --- B6_3: 평형으로 완화 (Part C의 핵심!) ---
def relax_to_equilibrium(x, x_eq, Gamma, dt):
    """x_temp = x_eq + (x - x_eq) * exp(-Γ*dt)"""
    return x_eq + (x - x_eq) * math.exp(-Gamma * dt)

# --- B6_4: 검증 - 큰 Γ*dt면 평형에 가까워지는가? ---
x_init = 0.5
x_eq = 0.1
x_after = relax_to_equilibrium(x_init, x_eq, Gamma=10.0, dt=1.0)
ans_B6_4 = abs(x_after - x_eq) < 0.01

# --- B6_5: 작은 Γ*dt면 거의 변하지 않는가? ---
x_small_Gamma = relax_to_equilibrium(x_init, x_eq, Gamma=0.01, dt=0.1)
ans_B6_5 = abs(x_small_Gamma - x_init) < 0.01

In [ ]:
grader.check('qB6', global_env=globals())

# Part C. Part A/B를 조합하여 BBN 시뮬레이션 완성하기

Part A에서 배운 **개념**과 Part B에서 만든 **함수들**을 조합하여 빅뱅 핵합성 시뮬레이션을 완성한다.

**핵심 아이디어:**
- Part B의 작은 빌딩 블록들 → Part C에서 큰 시스템으로 조립
- 왜 단순한 오일러 방법이 고온에서 실패하는지 이해
- 지수 완화(exponential relaxation)로 안정적인 적분기 구현

## C0. Part B에서 만든 함수들 확인하기

Part B에서 정의한 함수들이 제대로 작동하는지 확인하자.
이 함수들을 Part C에서 **그대로 재사용**한다!

**Part B에서 가져올 핵심 함수들:**
- `np_to_Xn()`: $n/p$ → $X_n$ 변환 (B1)
- `decay_fraction()`: 지수감쇠 계산 (B4)
- `t_of_T_B5()`: 시간-온도 관계 (B5)
- `Gamma_of_T_B5()`: 반응률-온도 관계 (B5)
- `relax_to_equilibrium()`: 지수 완화 (B6)

In [ ]:
# === Part B 함수 동작 확인 ===
print("=== Part B에서 가져온 함수들 테스트 ===")

# B1: 비율 변환
print(f"np_to_Xn(0.2) = {np_to_Xn(0.2):.4f}")  # 약 0.1667

# B4: 지수감쇠
print(f"decay_fraction(300, 879.4) = {decay_fraction(300, 879.4):.4f}")  # 약 0.71

# B5: 시간-온도, 반응률-온도
print(f"t_of_T_B5(2.0) = {t_of_T_B5(2.0):.4f}")  # 0.1845
print(f"Gamma_of_T_B5(2.0) = {Gamma_of_T_B5(2.0):.1f}")  # 96.0

# B6: 지수 완화
print(f"relax_to_equilibrium(0.5, 0.1, 10.0, 1.0) = {relax_to_equilibrium(0.5, 0.1, 10.0, 1.0):.4f}")

print("\n✅ 모든 Part B 함수가 정상 작동합니다!")

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt

# 상수(필요하면 바꿔보자)
A_default = 0.738          # [s * MeV^2]
gamma0_default = 3.0       # [s^-1 MeV^-5]
Q_default = 1.293          # [MeV] (n-p 질량차)
tau_n_default = 879.4      # [s] 중성자 수명

# 온도 격자(고온 -> 저온)
def make_T_grid(T_start=10.0, T_nuc=0.07, n_steps=400):
    return np.logspace(np.log10(T_start), np.log10(T_nuc), int(n_steps))

## C1. $t(T)=A/T^2$ — Part B5 함수 활용

Part B5에서 만든 `t_of_T_B5()` 함수를 사용하여 시간-온도 관계를 구현한다.

$$
t(T)=\frac{A}{T^2}
$$

**힌트:** Part B5에서 이미 구현했으므로, 그 함수를 그대로 호출하면 된다!

In [ ]:
# --- C1: Part B5의 t_of_T_B5를 활용! ---
def t_of_T(T, A=A_default):
    """t(T) = A/T^2 — Part B5에서 구현한 함수 활용"""
    out = ...  # 빈칸: t_of_T_B5(T, A)를 호출하거나 직접 A / (T * T)
    return out

In [ ]:
grader.check('qC1', global_env=globals())

## C2. $\Gamma(T)=\gamma_0 T^5$ — Part B5 함수 활용

Part B5에서 만든 `Gamma_of_T_B5()` 함수를 사용하여 반응률-온도 관계를 구현한다.

$$
\Gamma(T)=\gamma_0 T^5
$$

**힌트:** Part B5에서 이미 구현했으므로, 그 함수를 그대로 호출하면 된다!

In [ ]:
# --- C2: Part B5의 Gamma_of_T_B5를 활용! ---
def Gamma_of_T(T, gamma0=gamma0_default):
    """Γ(T) = γ₀T⁵ — Part B5에서 구현한 함수 활용"""
    out = ...  # 빈칸: Gamma_of_T_B5(T, gamma0)를 호출하거나 직접 gamma0 * (T ** 5)
    return out

In [ ]:
grader.check('qC2', global_env=globals())

## C3. 평형 $X_{n,\mathrm{eq}}(T)$ — Part B1 함수 활용

Part B1에서 만든 `np_to_Xn()` 함수를 사용하여 평형 중성자 분율을 계산한다.

**공식:**
$$
\frac{n}{p}=\exp(-Q/T),
\qquad
X_{n,\mathrm{eq}}=\frac{(n/p)}{1+(n/p)}
$$

**힌트:** 두 번째 식은 Part B1의 `np_to_Xn()`과 완전히 같은 형태이다!

In [ ]:
# --- C3: Part B1의 np_to_Xn를 활용! ---
def Xn_eq(T, Q=Q_default):
    """평형 중성자 분율 — Part B1의 비율 변환 함수 활용"""
    r = math.exp(-Q / T)  # n/p 비율
    Xeq = ...  # 빈칸: np_to_Xn(r)를 호출하거나 직접 r / (1 + r)
    return Xeq

In [ ]:
grader.check('qC3', global_env=globals())

## C4. 왜 단순 오일러 방법이 고온에서 실패하는가?

### 문제: Naive Euler의 불안정성

ODE $\frac{dX_n}{dt} = \Gamma \cdot (X_{n,\mathrm{eq}} - X_n)$ 를 단순 오일러로 풀면:

$$
X_{n,\text{next}} = X_n + \Gamma \cdot (X_{n,\mathrm{eq}} - X_n) \cdot dt = X_n \cdot (1 - \Gamma \cdot dt) + X_{n,\mathrm{eq}} \cdot (\Gamma \cdot dt)
$$

**고온(T ~ 10 MeV)에서:**
- $\Gamma \propto T^5$ 이므로 $\Gamma$가 매우 큼 (~ $10^5$ s$^{-1}$)
- $dt$가 작아도 $\Gamma \cdot dt \gg 1$이 될 수 있음
- $(1 - \Gamma \cdot dt)$ 항이 **음수**가 되어 $X_n$이 진동/발산!

### 해결책: 지수 완화 (Exponential Relaxation)

해석적 해를 사용하면:
$$
X_{n,\text{next}} = X_{n,\mathrm{eq}} + (X_n - X_{n,\mathrm{eq}}) \cdot e^{-\Gamma \cdot dt}
$$

**왜 안정적인가?**
- $e^{-\Gamma \cdot dt}$는 **항상 0과 1 사이**
- $\Gamma \cdot dt \gg 1$이면 $e^{-\Gamma \cdot dt} \to 0$ → $X_n \to X_{n,\mathrm{eq}}$ (빠르게 평형으로)
- $\Gamma \cdot dt \ll 1$이면 $e^{-\Gamma \cdot dt} \approx 1 - \Gamma \cdot dt$ → Euler와 동일

### 테일러 근사: $e^x \approx 1 + x$ (|x| ≪ 1일 때)

$$
e^x = 1 + x + \frac{x^2}{2!} + \frac{x^3}{3!} + \cdots \approx 1 + x \quad \text{(|x| ≪ 1)}
$$

따라서 $\Gamma \cdot dt \ll 1$이면:
$$
e^{-\Gamma \cdot dt} \approx 1 - \Gamma \cdot dt
$$

**결론:** 지수 완화는 Euler의 **일반화**이며, **항상 안정적**!

In [ ]:
# === C4: Naive Euler vs 지수 완화 비교 ===

# --- 1) Naive Euler (주의: 고온에서 불안정!) ---
def naive_euler_step(Xn, Xn_eq, Gamma, dt):
    """⚠️ 고온에서 불안정한 방법!"""
    return Xn + Gamma * (Xn_eq - Xn) * dt

# --- 2) 지수 완화 (Part B6의 relax_to_equilibrium과 동일!) ---
def exponential_relaxation(Xn, Xn_eq, Gamma, dt):
    """✅ 항상 안정적인 방법 (B6에서 배운 것!)"""
    return Xn_eq + (Xn - Xn_eq) * math.exp(-Gamma * dt)

# --- 3) 고온에서 Γ·dt 확인 ---
T_test = 5.0  # MeV (고온)
Gamma_test = Gamma_of_T_B5(T_test)  # Part B5 함수 사용!
dt_test = 0.01  # 초

Gamma_dt = Gamma_test * dt_test
print(f"T = {T_test} MeV에서:")
print(f"  Γ = {Gamma_test:.1f} s⁻¹")
print(f"  Γ·dt = {Gamma_dt:.2f}")

if Gamma_dt > 2:
    print(f"  ⚠️ Γ·dt > 2: Naive Euler는 진동/발산!")
elif Gamma_dt > 1:
    print(f"  ⚠️ Γ·dt > 1: Naive Euler가 불안정해질 수 있음")
else:
    print(f"  ✅ Γ·dt < 1: 두 방법 모두 안정")

# --- 4) O/X 문제: 테일러 근사 이해 확인 ---
# e^x ≈ 1 + x 는 |x| << 1 일 때만 좋은 근사이다

# 문제: e^0.01 ≈ 1.01 은 좋은 근사인가?
ans_C4_1 = ...  #@param ["O","X"]  # 힌트: |0.01| << 1

# 문제: e^(-10) ≈ 1 - 10 = -9 은 좋은 근사인가?
ans_C4_2 = ...  #@param ["O","X"]

In [ ]:
grader.check('qC4', global_env=globals())

## C5. 통합 스텝 함수 `step_Xn` — 모든 것을 조합!

지금까지 배운 **모든 함수**를 조합하여 한 스텝을 업데이트한다.

**사용하는 Part B 함수들:**
- C1: `t_of_T()` ← B5의 시간-온도 관계
- C2: `Gamma_of_T()` ← B5의 반응률
- C3: `Xn_eq()` ← B1의 비율 변환
- C4: `exponential_relaxation()` ← B6의 지수 완화

**업데이트 순서:**
1. 시간 간격 $dt$ 계산 (C1)
2. 반응률 $\Gamma$ 계산 (C2)
3. 평형값 $X_{n,\mathrm{eq}}$ 계산 (C3)
4. **지수 완화**로 약한 반응 적용 (C4/B6)
5. 중성자 붕괴 적용 (B4)

In [ ]:
# --- C5: 모든 Part A~C 개념을 통합한 스텝 함수 ---
def step_Xn(Xn, T_prev, T_now, A=A_default, gamma0=gamma0_default, Q=Q_default, tau_n=tau_n_default):
    """한 스텝 업데이트 — Part B의 모든 빌딩 블록 조합"""
    
    # === Step 1: 시간 간격 계산 (C1 = B5) ===
    t_prev = t_of_T(T_prev, A)
    t_now = t_of_T(T_now, A)
    dt = t_now - t_prev

    # === Step 2: 중간점 온도 (기하평균) ===
    T_mid = math.sqrt(T_prev * T_now)

    # === Step 3: 반응률 계산 (C2 = B5) ===
    lam_np = Gamma_of_T(T_mid, gamma0)           # n → p
    lam_pn = lam_np * math.exp(-Q / T_mid)       # p → n (상세평형)
    Gamma_wk = lam_np + lam_pn
    Xeq_mid = lam_pn / Gamma_wk                  # = np_to_Xn(exp(-Q/T))

    # === Step 4: 지수 완화 (C4 = B6) ★빈칸★ ===
    # 힌트: exponential_relaxation(Xn, Xeq_mid, Gamma_wk, dt) 사용!
    #       또는 직접: Xeq_mid + (Xn - Xeq_mid) * math.exp(-Gamma_wk * dt)
    Xn_temp = ...

    # === Step 5: 중성자 붕괴 적용 (B4) ===
    # 힌트: decay_fraction(dt, tau_n) 사용!
    Xn_next = Xn_temp * math.exp(-dt / tau_n)

    # === Step 6: 안전장치 ===
    Xn_next = max(0.0, min(1.0, Xn_next))
    return Xn_next

In [ ]:
grader.check('qC5', global_env=globals())

## C6. 결과 요약 $Y_p = 2X_n(T_{\mathrm{nuc}})$ — Part B3 함수 활용

Part B3에서 만든 `Yp_from_Xn_B3()` 함수와 동일한 공식이다!

$$
Y_p = 2X_n(T_{\mathrm{nuc}})
$$

**힌트:** Part B3의 함수를 그대로 호출해도 된다!

In [ ]:
# --- C6: Part B3의 Yp_from_Xn_B3를 활용! ---
def Yp_from_Xn(Xn_at_Tnuc):
    """헬륨 질량비 — Part B3에서 배운 공식"""
    Yp = ...  # 빈칸: Yp_from_Xn_B3(Xn_at_Tnuc) 또는 직접 2 * Xn_at_Tnuc
    return Yp

In [ ]:
grader.check('qC6', global_env=globals())

## C7. 시뮬레이션 실행 + 그래프

이 셀은 수정하지 않는다. Part B와 C에서 만든 함수들이 조합되어 전체 시뮬레이션이 실행된다!

**기대 결과:**
- 고온에서는 $X_n$이 평형(점선)을 따라감 → 반응이 빠름
- $\Gamma/H \approx 1$ 근처에서 평형이 깨짐 → **동결(freeze-out)**
- 이후에는 중성자 붕괴로 $X_n$이 천천히 감소
- 최종 $Y_p \approx 0.24$ (관측값과 유사!)

In [ ]:
def run_student_bbn(T_start=10.0, T_nuc=0.07, n_steps=600,
                    A=A_default, gamma0=gamma0_default, Q=Q_default, tau_n=tau_n_default):
    T = make_T_grid(T_start=T_start, T_nuc=T_nuc, n_steps=n_steps)
    Xn = Xn_eq(T[0], Q=Q)
    Xn_list = [Xn]
    t_list = [t_of_T(T[0], A)]
    Xeq_list = [Xn_eq(T[0], Q=Q)]
    for i in range(1, len(T)):
        Xn = step_Xn(Xn, float(T[i-1]), float(T[i]), A=A, gamma0=gamma0, Q=Q, tau_n=tau_n)
        Xn_list.append(Xn)
        t_list.append(t_of_T(T[i], A))
        Xeq_list.append(Xn_eq(T[i], Q=Q))

    Xn_at_Tnuc = float(Xn_list[-1])
    Yp = Yp_from_Xn(Xn_at_Tnuc)
    return {
        "T": T,
        "t": np.array(t_list),
        "Xn": np.array(Xn_list),
        "Xn_eq": np.array(Xeq_list),
        "Yp": float(Yp),
    }

run_student = run_student_bbn()
print("Yp(토이) =", round(run_student["Yp"], 4))

# 그래프 2개만 간단히: (1) Xn vs t, (2) n/p vs T
T = run_student["T"]
Xn = run_student["Xn"]
Xeq = run_student["Xn_eq"]

eps = 1e-12
np_ratio = np.maximum(eps, Xn) / np.maximum(eps, 1.0 - Xn)
np_eq = np.maximum(eps, Xeq) / np.maximum(eps, 1.0 - Xeq)

plt.figure(figsize=(7.0,4.8))
plt.plot(run_student["t"], Xn, label="실제(토이)")
plt.plot(run_student["t"], Xeq, "--", label="평형(점선)")
plt.xscale("log")
plt.xlabel("시간 t [초]")
plt.ylabel("중성자 분율 Xn")
plt.title("Xn(t)")
plt.grid(True, which="both", alpha=0.3)
plt.legend()
plt.show()

plt.figure(figsize=(7.0,4.8))
plt.plot(T, np_ratio, label="실제(토이)")
plt.plot(T, np_eq, "--", label="평형(점선)")
plt.xscale("log"); plt.yscale("log"); plt.gca().invert_xaxis()
plt.xlabel("온도 T [MeV]")
plt.ylabel("n/p")
plt.title("n/p(T)")
plt.grid(True, which="both", alpha=0.3)
plt.legend()
plt.show()

In [ ]:
grader.check('qC7', global_env=globals())

## C8. 그래프 해석 — 동결 과정 말로 정리하기

Part A7-A8에서 배운 무차원비 $\Gamma/H$와 그래프 해석을 적용해보자!

In [ ]:
# --- C8: 그래프 해석 (A7-A8 개념 적용) ---

# 고온에서 Xn이 평형(점선)을 잘 따라가는 이유는?
# A: 팽창(H)이 반응(Γ)보다 더 빠르기 때문
# B: 반응(Γ)이 팽창(H)보다 더 빠르기 때문
ans_C8_1 = ...  #@param ["A","B"]  # 힌트: Γ/H > 1 → 평형 유지

# Γ/H가 1보다 작아지면?
# A: 평형이 깨지기 쉬움 (동결)
# B: 평형이 더 잘 유지됨
ans_C8_2 = ...  #@param ["A","B"]  # 힌트: A7 복습!

# 동결 이후 Xn이 천천히 줄어드는 주요 이유는?
# A: 중성자 붕괴 (B4에서 배움)
# B: 다시 평형으로 회복
ans_C8_3 = ...  #@param ["A","B"]  # 힌트: τ_n ≈ 879초

In [ ]:
grader.check('qC8', global_env=globals())

## C9. 최종 점검: 오늘의 질문에 답해보자!

시뮬레이션을 완성했다! 이제 처음에 던진 **세 가지 질문**에 스스로 답할 수 있는지 확인해보자.

---

### 질문 1: 왜 고온에서는 n/p가 "평형"을 따라가나?

**힌트:** A7에서 배운 $\Gamma/H$ 무차원비, C8에서 본 그래프를 떠올려보자.

---

### 질문 2: 왜 어떤 시점부터 평형이 깨지고 "동결(freeze-out)"되나?

**힌트:** 온도가 내려가면 $\Gamma$와 $H$가 어떻게 변하는지 생각해보자.

---

### 질문 3: 왜 마지막에 $Y_p \approx 2X_n$이 되는가?

**힌트:** A3-A4에서 배운 제한 반응물, 핵자수 보존을 떠올려보자.

In [ ]:
# === C9: 세 가지 질문 최종 점검 ===

# --- 질문 1: 왜 고온에서는 n/p가 "평형"을 따라가나? ---
# A: 반응(Γ)이 우주 팽창(H)보다 빨라서, 평형을 유지할 시간이 있기 때문
# B: 우주 팽창이 너무 빨라서, 입자들이 움직이지 못하기 때문
ans_C9_Q1 = ...  #@param ["A","B"]

# 질문 1 심화: Γ/H > 1일 때 어떤 일이 일어나는가?
# A: 반응이 빨라서 평형 유지
# B: 팽창이 빨라서 동결
ans_C9_Q1_detail = ...  #@param ["A","B"]

# --- 질문 2: 왜 어떤 시점부터 평형이 깨지고 "동결"되나? ---
# A: 온도가 내려가면 Γ ∝ T⁵ 가 급격히 감소하여 Γ/H < 1이 되기 때문
# B: 온도가 내려가면 중성자가 모두 붕괴해서
ans_C9_Q2 = ...  #@param ["A","B"]

# 질문 2 심화: 동결 온도는 대략 몇 MeV인가? (그래프에서 평형이 깨지는 지점)
# A: 약 10 MeV
# B: 약 1 MeV (실제 약 0.7-1 MeV)
# C: 약 0.01 MeV
ans_C9_Q2_detail = ...  #@param ["A","B","C"]

# --- 질문 3: 왜 Yp ≈ 2Xn 인가? ---
# A: He-4 하나에 중성자 2개가 들어가고, 중성자가 제한 반응물이기 때문
# B: He-4 하나에 양성자 2개가 들어가고, 양성자가 제한 반응물이기 때문
ans_C9_Q3 = ...  #@param ["A","B"]

# 질문 3 심화: Xn = 0.125일 때 Yp는?
ans_C9_Q3_calc = ...  # 빈칸

# --- 종합: 이 시뮬레이션에서 배운 핵심 물리 ---
# 아래 설명 중 올바른 것은?
# A: 초기 우주에서 모든 핵종이 동시에 생성되었다
# B: 고온 → 저온으로 식으면서, n/p 비율이 동결되고, 그 비율에 따라 He-4 양이 결정된다
ans_C9_summary = ...  #@param ["A","B"]

print("축하합니다! BBN 시뮬레이션의 핵심 물리를 이해했습니다!")

In [ ]:
grader.check('qC9', global_env=globals())

# 제출 전 최종 점검

- 아래 셀로 전체 문항을 한 번에 확인하자.

In [ ]:
grader.check_all()